#Fall 2024 AIN433 Assignment 2
**Student:** Mehmet Ertaş - 2200765035

**Instructor:** Ali Seydi Keçeli

**TA:** Sibel Kapan

In [ ]:
!pip install -U opencv-contrib-python==3.4.2.16
!pip install scikit-image

Requirement already up-to-date: opencv-contrib-python==3.4.2.16 in c:\users\galip\anaconda3.5\lib\site-packages (3.4.2.16)
Requirement not upgraded as not directly required: numpy>=1.11.3 in c:\users\galip\anaconda3.5\lib\site-packages (from opencv-contrib-python==3.4.2.16) (1.14.3)


distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from skimage.measure import compare_ssim as ssim
import glob
import os
import random
from PIL import Image

In [ ]:
def testSURF(img1):
  compute = cv2.xfeatures2d.SURF_create()
  keypoints, descriptors = compute.detectAndCompute(img1, None)
  image_with_keypoints = cv2.drawKeypoints(img1, keypoints, None, color=(0, 255, 0), flags=0)

  fig = plt.figure()
  fig.set_size_inches(18,10)
  plt.imshow(cv2.cvtColor(image_with_keypoints, cv2.COLOR_BGR2RGB))


img = cv2.imread("C:/Users/galip/Downloads/Panaromic_Dataset/p1/cyl_image00.png", cv2.IMREAD_COLOR)
testSURF(img)

In [ ]:
def compute_homography(src_pts, dst_pts):
  N = src_pts.shape[0]

  H=[]
  src_array = np.asarray(src_pts)
  dst_array = np.asarray(dst_pts)

  for n in range(N):
    src = src_array[n]
    H.append(-src[0])
    H.append(-src[1])
    H.append(-1)
    H.append(0)
    H.append(0)
    H.append(0)

  H = np.asarray(H)
  H1 = H.reshape(2*N,3)

  H2 = np.zeros([2*N, 3], dtype=int)
  for i in range(0,2*N,2):
    H2[i:i+2,0:i+3] = np.flip(H1[i:i+2,0:i+3], axis=0)

  H2 = np.asarray(H2)
  H3 = np.concatenate((H1, H2), axis=1)

  H4=[]
  for n in range(N):
    src = src_array[n]
    dst = dst_array[n]

    H4.append(src[0]*dst[0])
    H4.append(src[1]*dst[0])
    H4.append(dst[0])
    H4.append(src[0]*dst[1])
    H4.append(src[1]*dst[1])
    H4.append(dst[1])

  H4 = np.asarray(H4)
  H4 = H4.reshape(2*N,3)

  H5 = np.concatenate((H3, H4), axis=1)
  H8 = np.matmul(np.transpose(H5), H5)

  w, v = np.linalg.eig(H8)
  minimum = w.min()
  for i in range(len(w)):
    if w[i] == minimum:
      a = v[:, i]

  a = np.asarray(a)
  a = a.reshape(3,3)
  a = a/a[2,2]

  return a


def apply_homography(test, H):
  dst_output = []
  N = test.shape[0]

  for row in test:
    input = np.matrix([row[0,0], row[0,1], 1])
    input = input.transpose()
    mapped_pts = np.matmul(H, input)
    dst_output.append(mapped_pts[0]/mapped_pts[2])
    dst_output.append(mapped_pts[1]/mapped_pts[2])

  dst_output = np.asarray(dst_output)
  dst_output = dst_output.reshape(N, 2)

  return dst_output

def genMatchPairs(img1, img2):
  compute = cv2.ORB_create(nfeatures=3000)
  kp1, des1 = compute.detectAndCompute(cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY), None)
  kp2, des2 = compute.detectAndCompute(cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY), None)
  bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
  matches = bf.match(des1, des2)
  matches = sorted(matches, key = lambda x:x.distance)

  pts1 = np.zeros((250, 2))
  pts2 = np.zeros((250, 2))
  for i in range(250):
    pts1[i,:] = kp1[matches[i].queryIdx].pt
    pts2[i,:] = kp2[matches[i].trainIdx].pt

  return pts1, pts2, matches[:250], kp1, kp2

In [ ]:
def RANSAC(Xs, Xd, max_iter, eps):
  H = np.zeros([3,3])


  inliers_ids = []
  inliers_counts = []

  n = Xs.shape[0]

  iter = 0
  while iter < max_iter:
    inliers_id = []
    pts_index = random.sample(range(0, n), 4)

    Xs_new = []
    Xd_new = []

    for pt in range(4):
      Xs_new.append(Xs[pts_index[pt]][:])
      Xd_new.append(Xd[pts_index[pt]][:])

    Xs_new = np.asarray(Xs_new)
    Xd_new = np.asarray(Xd_new)
    Xs_new = np.asmatrix(Xs_new)
    Xd_new = np.asmatrix(Xd_new)

    H = compute_homography(Xs_new, Xd_new)

    Xs = np.asmatrix(Xs)
    Xd_predicted = apply_homography(Xs, H)

    for i in range(n):
      SSD = ((round(Xd_predicted[i][0]) - int(Xd[i, 0]))**2 + (round(Xd_predicted[i][1]) - int(Xd[i, 1]))**2)

      if SSD < eps:
        if i not in inliers_id:
          inliers_id.append(i)

    inliers_ids.append(inliers_id)
    inliers_counts.append(len(inliers_id))

    iter += 1

  largest_count_index = inliers_counts.index(max(inliers_counts))
  best_inliers_id = inliers_ids[largest_count_index]

  Xs_inliers = []
  Xd_inliers = []
  for i in best_inliers_id:
    Xs_inliers.append(Xs[i][:])
    Xd_inliers.append(Xd[i][:])

  Xs_inliers = np.asarray(Xs_inliers)
  Xd_inliers = np.asarray(Xd_inliers)
  Xs_inliers = np.asmatrix(Xs_inliers)
  Xd_inliers = np.asmatrix(Xd_inliers)

  H = compute_homography(Xs_inliers, Xd_inliers)

  return best_inliers_id, H

In [ ]:
def warp_img(src_img, H, dst_img_size):
    M, N = dst_img_size
    dst_img = np.zeros((M, N, src_img.shape[2]), dtype=src_img.dtype)

    H_inv = np.linalg.inv(H)
    for i in range(N):
        for j in range(M):
            src_coords = np.dot(H_inv, np.array([i, j, 1]))
            src_coords /= src_coords[2]

            src_x, src_y = src_coords[0], src_coords[1]

            src_x = int(round(src_x))
            src_y = int(round(src_y))


            if 0 <= src_y < src_img.shape[0] and 0 <= src_x < src_img.shape[1]:
                dst_img[j, i] = src_img[src_y, src_x]
    return dst_img

def binary_mask(img):
    mask = (img[:, :, 0] > 0) | (img[:, :, 1] > 0) | (img[:, :, 2] > 0)
    mask = mask.astype("int")
    return mask

In [ ]:
def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            img = cv2.imread(file_path)
            if img is not None:
                images.append(img)
            else:
                print(f"Warning: Could not load image {file_path}")
    return images

In [ ]:
folder_path = "C:/Users/galip/Downloads/Panaromic_Dataset/p1"

images = load_images_from_folder(folder_path)
print(f"Loaded {len(images)} images.")

Loaded 16 images.


In [ ]:
for idx, img in enumerate(images):
    if img is None or img.size == 0:
        print(f"Image {idx} is empty or could not be loaded.")
    else:
        print(f"Image {idx} loaded successfully with shape {img.shape}.")

Image 0 loaded successfully with shape (480, 479, 3).
Image 1 loaded successfully with shape (480, 479, 3).
Image 2 loaded successfully with shape (480, 479, 3).
Image 3 loaded successfully with shape (480, 479, 3).
Image 4 loaded successfully with shape (480, 479, 3).
Image 5 loaded successfully with shape (480, 479, 3).
Image 6 loaded successfully with shape (480, 479, 3).
Image 7 loaded successfully with shape (480, 479, 3).
Image 8 loaded successfully with shape (480, 479, 3).
Image 9 loaded successfully with shape (480, 479, 3).
Image 10 loaded successfully with shape (480, 479, 3).
Image 11 loaded successfully with shape (480, 479, 3).
Image 12 loaded successfully with shape (480, 479, 3).
Image 13 loaded successfully with shape (480, 479, 3).
Image 14 loaded successfully with shape (480, 479, 3).
Image 15 loaded successfully with shape (480, 479, 3).


In [ ]:
def stitch_multiple(imgs):
    stitched_img = np.pad(imgs[0], pad_width=((400, 400), (400, 400), (0, 0)), mode = "constant")

    for i in range(1, len(imgs)):
        current_img = imgs[i]
        print(i)
        if stitched_img.shape[:2] != current_img.shape[:2]:
            current_img = resize_to_match(current_img, stitched_img.shape[:2])

        pts1, pts2, matches1to2, kp1, kp2 = genMatchPairs(current_img, stitched_img)
        inliers_idx, H = RANSAC(pts1, pts2, 500, 50)

        warped_img = warp_img(current_img, H, [stitched_img.shape[0], stitched_img.shape[1]])

        stitched_mask = 1 - binary_mask(warped_img)
        stitched_mask = np.stack((stitched_mask,) * 3, -1)
        stitched_img = np.multiply(stitched_img, stitched_mask) + warped_img

        stitched_img = crop_to_content(stitched_img)

    return stitched_img


In [ ]:
final_stitch = stitch_multiple(images)
plt.figure(figsize=(20, 10))
plt.imshow(cv2.cvtColor(final_stitch.astype("uint8"), cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()


1
2


error: OpenCV(3.4.2) c:\projects\opencv-python\opencv\modules\imgproc\src\color.hpp:253: error: (-215:Assertion failed) VScn::contains(scn) && VDcn::contains(dcn) && VDepth::contains(depth) in function 'cv::CvtHelper<struct cv::Set<3,4,-1>,struct cv::Set<1,-1,-1>,struct cv::Set<0,2,5>,2>::CvtHelper'


In [ ]:
def crop_to_content(img):
    if len(img.shape) != 3 or img.shape[2] != 3:
        raise ValueError("Input image must have 3 color channels (BGR).")
    gray = cv2.cvtColor(img.astype("uint8"), cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
    x, y, w, h = cv2.boundingRect(thresh)
    cropped_img = img[y:y+h, x:x+w]

    return cropped_img

In [1]:
def resize_to_match(img, target_size):
    return cv2.resize(img, (target_size[1], target_size[0]))
